In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history' # template 안에 memory가 history를 저장하도록 한 곳
)

template = """
You are a helpful AI talking to a human

{chat_history}
Human: {question}
You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, # chain의 prompt 로그를 확인할 수 있음
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human


Human: My name is Nico
You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [20]:
chain.predict(question="I live is Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human

Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live is Seoul
You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you with anything related to Seoul?"

In [21]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human

Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live is Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you with anything related to Seoul?
Human: What is my name?
You:


> Finished chain.


'Your name is Nico.'

In [18]:
memory.load_memory_variables({})

{'history': "System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them. Nico mentions that they live in Seoul, and the AI responds by providing information about Seoul's culture, architecture, and attractions. The AI then asks Nico what they enjoy most about living in Seoul.\nHuman: What is my name?\nAI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."}

In [17]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})